# Music Recommendation

## Data Preparation

In [21]:
import pandas as pd
import os
# Matrix Factorization
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [2]:
fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [3]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
# 첫번째 유저가 무슨 가수의 노래를 듣는지 확인
condition = data['user_id']== data.loc[0, 'user_id']
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


## Data Exploration

In [6]:
# 유저 수
data['user_id'].nunique()

358868

In [7]:
# 아티스트 수
data['artist'].nunique()

291346

In [8]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [9]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계 (48.9 +- 8.5)
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [10]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

In [14]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


In [15]:
# Indexing
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [16]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


In [18]:
# 1회만 플레이한 데이터의 비율
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


## Model

- Matrix Factorization 사용: 성능이 좋고, Scalability 좋음.
   
![MF](https://miro.medium.com/max/700/1*Zhm1NMlmVywn0G18w3exog.png)
- m = 4, n = 2, k = 2   
- Rating Matrix를 User Matrix, Item Matrix로 분해  $U_i \bullet I_j = M_{ij}$   
[Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)

In [19]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

In [22]:
# als(AlternatingLeastSquares) model
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, # 몇 차원
                                    regularization=0.01, # overfitting 막기 위해 정규화 값 얼마나?
                                    use_gpu=False, # GPU 사용여부
                                    iterations=15, # =epochs (몇번반복학습?)
                                    dtype=np.float32)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [26]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [27]:
zimin_vector

array([ 1.1570214 ,  0.02686394, -0.15401645, -0.49445274, -0.8282687 ,
       -0.33007517, -1.0712595 ,  0.60915333, -0.03187579,  0.21944608,
        0.31718677, -0.23771913,  1.2951766 , -0.08935475,  1.0547856 ,
       -0.27592534,  0.24090964, -1.1375108 , -0.97608155, -0.28403813,
       -0.70830184, -0.86163294,  1.7059675 ,  0.16802809,  0.41512692,
        0.31863773, -1.2406166 ,  0.31150156, -0.33088112,  0.03297267,
       -0.67623794,  0.8209415 ,  0.07001998, -0.17666385,  0.11702576,
       -0.636508  , -0.8801349 ,  0.81270725, -0.8375869 , -0.24226378,
       -0.4371584 ,  0.5728118 ,  0.6926235 , -0.5058149 , -0.50123477,
        0.03221415,  1.0634593 ,  0.22710106, -0.80371493,  1.4200865 ,
        0.06105004, -0.4231211 , -1.3611962 , -1.4291625 ,  0.12053037,
        0.07001907,  0.6893869 ,  0.27891347,  0.3036267 ,  0.14924853,
        0.49884105,  0.10597389, -0.18489273, -0.64927393,  0.80648595,
       -0.55281526,  0.11991221,  0.05510005,  0.5200369 , -0.00

In [28]:
black_eyed_peas_vector

array([ 0.02263928,  0.01964312,  0.00601346,  0.0022747 , -0.00091317,
        0.01139034,  0.00796402,  0.00471191,  0.0098262 ,  0.02321514,
       -0.00224832,  0.00629459,  0.0278347 ,  0.00485856,  0.01082614,
       -0.01013275,  0.01761498, -0.01588827, -0.00712409, -0.01025276,
       -0.00627067, -0.01380614,  0.03371247,  0.01449199,  0.02065928,
        0.00420329, -0.00499454,  0.00670126,  0.01560325,  0.00281055,
       -0.00239089,  0.01642796,  0.00526629, -0.00263562,  0.00701782,
        0.00103163, -0.01135224,  0.03703856,  0.00638132, -0.00078885,
        0.00612434,  0.00836974,  0.02249718,  0.00462692,  0.01456593,
        0.00613065,  0.01325615,  0.00531191, -0.02528673,  0.02009623,
        0.01107173, -0.00538398, -0.01287771, -0.00561696,  0.00843326,
        0.02388594,  0.00494691,  0.00727508, -0.00708525,  0.01074198,
        0.01280966,  0.01904582,  0.01128092,  0.00204782, -0.00076907,
       -0.0003484 ,  0.00420742,  0.00676227,  0.00660346, -0.00

In [29]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.5021733

In [30]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.3212604

In [31]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.12286319),
 (277, 0.120932266),
 (28, 0.12054405),
 (5, 0.119844005),
 (217, 0.11941875),
 (473, 0.11879979),
 (247, 0.11849554),
 (490, 0.118365094),
 (418, 0.118314795),
 (910, 0.11701688),
 (1018, 0.11623705),
 (268, 0.11617967),
 (694, 0.115647756),
 (55, 0.11532614),
 (75, 0.11525843)]

In [32]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'placebo',
 'the beatles',
 'oasis',
 'u2',
 'nirvana',
 'the smashing pumpkins',
 'pink floyd',
 'foo fighters',
 'arctic monkeys',
 'queen']

In [33]:
# 반복 확인
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [34]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'jay-z',
 'nas',
 '50 cent',
 'the game',
 'dr. dre',
 'ice cube',
 'common',
 'dmx']

In [35]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'leona lewis',
 'christina aguilera',
 'nelly furtado',
 'pink']

In [37]:
get_similar_artist('beyonce')

['beyonce',
 'cherish ft. yung joc',
 'brit & alex',
 'the fugees',
 'am2pm',
 'denis the menace',
 'jessie malakouti',
 'janet leon',
 'global deejays feat. technotronic',
 'andy aguilera']

In [38]:
# 유저에게 아티스트 추천
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.46959877),
 (369, 0.4647918),
 (550, 0.46133402),
 (627, 0.42326552),
 (1800, 0.4229226),
 (354, 0.41446793),
 (391, 0.4105907),
 (2249, 0.40945894),
 (355, 0.40512946),
 (901, 0.39276266),
 (274, 0.3880812),
 (24, 0.38076714),
 (621, 0.3805194),
 (944, 0.37943423),
 (5555, 0.37537357),
 (1777, 0.37080353),
 (618, 0.3707436),
 (564, 0.36229622),
 (358, 0.35467806),
 (28, 0.35360342)]

In [39]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'justin timberlake',
 'britney spears',
 'maroon 5',
 'lady gaga',
 'nelly furtado',
 'christina aguilera',
 'katy perry',
 'madonna',
 'pink',
 'michael jackson',
 'jack johnson',
 'alicia keys',
 'avril lavigne',
 'timbaland',
 'mariah carey',
 'the pussycat dolls',
 'kanye west',
 'kelly clarkson',
 'the killers']

In [40]:
# rihanna 추천한 이유 확인 (추천에 기여한 정도)
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [41]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.2176363614898854),
 ('black eyed peas', 0.13956450001362308),
 ('coldplay', 0.05862816535109436),
 ('jason mraz', 0.05679863992261924),
 ('maroon5', 0.00013405122478076344)]

- 유저 아티스트에 대한 메타정보 반영 쉽지 않음. 연령대별로 매우 다향
- 유저가 언제 음악을 들은건지 반영이 어려움